In [13]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn import datasets

In [14]:
pwd = os.getcwd() #capturing the present working directory

In [15]:
#Converting the data as it was recieved into 
read_file = pd.read_csv (pwd + r'\Data\adult.csv') # by using pwd + I am able to shorten the path required.
df = pd.DataFrame(read_file)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:

df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
salary            object
dtype: object

In [17]:
df_mod = df.copy()
df_mod.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


I'm first going to drop any columns I know I won't need at this point.

In [18]:
col_to_drop = ['workclass', 'fnlwgt', 'education-num', 'occupation', 'race', 'capital-gain', 'capital-loss', 'country']
col_to_drop

['workclass',
 'fnlwgt',
 'education-num',
 'occupation',
 'race',
 'capital-gain',
 'capital-loss',
 'country']

In [19]:
df_mod = df_mod.drop(columns=col_to_drop)
df_mod.head()

,age,education,marital-status,relationship,sex,hours-per-week,salary
0,39,Bachelors,Never-married,Not-in-family,Male,40,<=50K
1,50,Bachelors,Married-civ-spouse,Husband,Male,13,<=50K
2,38,HS-grad,Divorced,Not-in-family,Male,40,<=50K
3,53,11th,Married-civ-spouse,Husband,Male,40,<=50K
4,28,Bachelors,Married-civ-spouse,Wife,Female,40,<=50K


With this result I can see that I still need to sort the data into more of a binary approach if the clustering is going to work.

Now that I have made a copy of the edited dataset I can start to pick points of interest.

In [20]:
df_mod_copy = df_mod.copy()
col_to_drop=['education', 'marital-status', 'relationship', 'salary']
df_mod_copy = df_mod_copy.drop(columns=col_to_drop)
df_mod_copy.head()

,age,sex,hours-per-week
0,39,Male,40
1,50,Male,13
2,38,Male,40
3,53,Male,40
4,28,Female,40


As you can see I am now left with three columns or points of interest. age, sex and hours-per-week. I will be able to build the scatter graph showing how age relates to how many hours people work. But I also have the possibility of seperating the data into two scatter graphs which would also show not just how age affects, but also sex.

Now I am going to split this data into Male and Female and change the value of the cells to 1 or 0.

In [21]:
final_result = df_mod_copy.copy()

for x in final_result.index:
    if final_result.loc[x, "sex"] == "Male":
        final_result.at[x, "sex"] = 1
    else:
        final_result.at[x, "sex"] = 2
        """In this case we do not need to check whether the other 
        value would be anything but what we expected (Male/ Female) because the 'sex'
        column has no missing data or unexpected values. However, in other situations
        I may have had to check for unknowns."""
        
final_result.head()


,age,sex,hours-per-week
0,39,1,40
1,50,1,13
2,38,1,40
3,53,1,40
4,28,2,40


In [22]:
df_male = final_result.copy()

for x in df_male.index:
    if df_male.loc[x, "sex"] == 2:
        df_male.drop(x, inplace = True)
        """In this case we do not need to check whether the other 
        value would be anything but what we expected (Male/ Female) because the 'sex'
        column has no missing data or unexpected values. However, in other situations
        I may have had to check for unknowns."""
        
df_male.head()


,age,sex,hours-per-week
0,39,1,40
1,50,1,13
2,38,1,40
3,53,1,40
7,52,1,45


In [23]:
df_female = final_result.copy()

for x in df_female.index:
    if df_female.loc[x, "sex"] == 1: 
        df_female.drop(x, inplace = True)
        """In this case we do not need to check whether the other 
        value would be anything but what we expected (Male/ Female) because the 'sex'
        column has no missing data or unexpected values. However, in other situations
        I may have had to check for unknowns."""
        
df_female.head()

,age,sex,hours-per-week
4,28,2,40
5,37,2,40
6,49,2,16
8,31,2,50
12,23,2,30


In [24]:
writer = pd.ExcelWriter(pwd + "/Data/data_output.xlsx", engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = 'adult_data')
df_mod.to_excel(writer, sheet_name = 'data_FirstEdit')
df_mod_copy.to_excel(writer, sheet_name = 'data_SecondEdit')
df_male.to_excel(writer, sheet_name = 'Final_MaleData')
df_female.to_excel(writer, sheet_name = 'Final_FemaleData')
final_result.to_excel(writer, sheet_name = 'Final_Result')
writer.save()
